In [1]:
from pathlib import Path
from utils import io as IO
from utils.device import DEVICE
from utils import backbone as BKB
from utils import head as HEAD
from PIL import Image
from utils.transforms import TRANSFORM, TARGET_TRANSFORM 
import torch
import pandas as pd
import cv2
from tqdm import tqdm
from utils.dataset import TorchDataset
from torch.utils.data import DataLoader
from utils.model import SegmentationModel

c:\Users\bldr1\miniconda3\envs\ike\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\bldr1\miniconda3\envs\ike\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
LABELS_PATH = "data/ISG-Guywire_Ground_Seg/"
IMAGES_PATH = ""

# get images and labels paths
images_files = [x for x in Path(LABELS_PATH).glob("*.JPG")]
labels_files = [x for x in Path(LABELS_PATH).glob("*.png")]

print(f"Number of images {len(images_files)}")
print(f"Number of labels {len(labels_files)}")

images_files = pd.DataFrame(images_files, columns = ["image_path"])
labels_files = pd.DataFrame(labels_files, columns = ["label_path"])

images_files["stem"] = images_files["image_path"].apply(lambda x: x.stem)
labels_files["stem"] = labels_files["label_path"].apply(lambda x: x.stem)

consolidation = images_files.merge(labels_files, how="inner", on="stem")
print(f"Dataset size {len(consolidation)}")

Number of images 419
Number of labels 419
Dataset size 419


## Overlay images and GT to understand labels

In [3]:
# you can skip this cell
run_overlay = False
if run_overlay:
    overlay_dir = "data/overlay"
    Path(overlay_dir).mkdir(parents=True, exist_ok=True)
    for i, row in tqdm(consolidation.iterrows(), total=len(consolidation)):
        image_path = row["image_path"]
        label_path = row["label_path"]

        image = IO.load_image(image_path)
        mask = IO.load_image(label_path)

        overlay = cv2.addWeighted(image,0.9,mask,0.1,0)
        dst_path = Path(overlay_dir).joinpath(f"{row['stem']}.png")
        # the channels are shifted but it's enough to understand the data
        cv2.imwrite(str(dst_path), cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))
    

## Split data and create dataloaders

In [4]:
dataset = consolidation.copy().sample(frac=1, random_state=324)
train_set = dataset.iloc[:int(len(dataset) * 0.8), :]
val_set = dataset.iloc[int(len(dataset) * 0.8):, :]

print(f"training set: {len(train_set)}")
print(f"val set: {len(val_set)}")

training set: 335
val set: 84


In [5]:
train_ds = TorchDataset(train_set, transform=TRANSFORM, target_transform=TARGET_TRANSFORM)
val_ds = TorchDataset(val_set, transform=TRANSFORM, target_transform=TARGET_TRANSFORM)

train_dataloader = DataLoader(train_ds, shuffle=True, pin_memory=True, batch_size=8)
val_dataloader = DataLoader(val_ds, shuffle=False, pin_memory=True, batch_size=8)

## Initialize model

In [6]:
# 2 classes, background and guy wire
# backbone size S -> 384 
backbone = BKB.get_backbone(size="s")
head = HEAD.get_head(n_classes=2, input_dims=384, layer_sizes=(64, 64))

backbone.eval()
backbone.to(DEVICE)

head.to(DEVICE)

seg_model = SegmentationModel(backbone=backbone, head=head)

Using cache found in C:\Users\bldr1/.cache\torch\hub\facebookresearch_dinov2_main
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.0.0+cu118 with CUDA 1108 (you have 2.0.0+cpu)
    Python  3.10.10 (you have 3.10.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [9]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(seg_model.parameters(), lr=0.001)

def train_one_epoch():

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(train_dataloader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = seg_model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
    return running_loss / len(train_dataloader)

In [11]:
train_one_epoch()

TypeError: Unexpected type <class 'numpy.ndarray'>